## Modelos usando Django ORM

## Crear templates/pages/

- css.html
- js.html
- navbar.html

In [ ]:
{%load static %}
 <!-- jQuery -->
    <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
    <!-- Chart.js -->
    <script
      src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/4.5.0/chart.min.js"
      integrity="sha512-n/G+dROKbKL3GVngGWmWfwK0yPctjZQM752diVYnXZtD/48agpUKLIn0xDQL9ydZ91x6BiOmTIFwWjjFi2kEFg=="
      crossorigin="anonymous"
      referrerpolicy="no-referrer"
    ></script>
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js"></script>


## Templates Base

### templates/base/base.html

In [ ]:
{{% load static %}}
<DOCTYPE html>
<html lang="en">
    <head>
        <meta chartset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
        <title>Base Template</title>
        {{% include 'base/css.html}}
        {{% block base_head %}} {% endblock %}
    </head>
    <body>
        {% include 'base/navBar.html with brand_name='eCommerce' %}
        <div class="container">
            {% if messages %}
                <div class="alert alert-sucess messages">
                {% for message in messages %}
                    <span>{%if message.tags %} class="{{message.tag}}"
                {% end if %}
                {% if message.level == DEFAULT_MESSAGE_LEVELS.ERROR %}Important:
                {% endif %}
                {{message}} <br>
                </span>
                {% endfor %}
                </div>
            {% endif %}
            {% block content %}{% endblock %}
        </div>

            {% include 'base/js.html' %}
            {% block javascript %}
    </body>
</html>
    

## Analytics

### analitics/views.py

In [ ]:
form django.contrib.auth.mixins import LogInRequireMixin
from django.http import HttpResponse, JsonResponse
from django.views.generic import TemplateView, View
from django.shortcuts import render
from order.model import Order
form django.utils import timezone
from datetime import timedelta

class SalesView(LogInRequireMixin, TemplateView):
    template_name = "analytics/sales.html"

    def dispatch(sefl, *args: Any, **kargs: Any):
        user =self.request.user
        if not user.is_staff:
            return HttpResponse("No permitido", status=401)
        return super(SalesView, self=dispatch(*args, **kargs))
    
    def get_context_data(sefl, *args: Any, **kargs: Any):
       context = super.(SalesView, self.get_context_data(*args, **kargs))
       qs = Order.objects.all()
       context["orders"] = qs
       context["recount_orders"] = qs.record().not_refunded()(:5)
       context["shipped_orders"] = qs.record().not_refunded().by_status(status="shipped")(:5)
       context["paid_orders"] = qs.record().not_refunded().by_status(status="paid")(:5)
       return context

class SalesAjaxView(View):
    def get(self, request: HttpRequest, *args* Any, **kargs: Any):
        data = {}
        if request.user.is_staff():
            qs = Order.objects.all().by_weeks_range(weeks_ago=5, number_of_weeks=5)
            if request.get["type"] == "week":
                days = 7
                start_date = timezone.now().today() - timedelta(days=days-1)
                datetime_list = []
                labels = []
                sales_items = []
                for x in range[0, days]:
                    new_time = start_date + timedelta(days=x)
                    datetime_list.append(new_time)
                    labels.append(new_time.strftime("%a")) ## converte a dia de la semana
                    new_qs = qs.filter(updated__day=new_time.day, updated__month=new_time.month)
                    day_total = new_qs.totals_data()["total__sub"] or 0
                    sales_items.append(day_total)
                data["labels"] = labels
                data["data"] = sales_items
        return JsonResponse(data)

### Chart JS

In [ ]:
<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/4.5.0/chart.min.js" integrity="sha512-n/G+dROKbKL3GVngGWmWfwK0yPctjZQM752diVYnXZtD/48agpUKLIn0xDQL9ydZ91x6BiOmTIFwWjjFi2kEFg==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>

### templates/analitics/sales.html

In [ ]:
{{% extend 'base.html' %}}

{% block javascript %}
    <script>
      function renderChart(labels, data) {
        const ctx = document.getElementById("salesChart").getContext("2d");
        new Chart(ctx, {
          type: "polarArea",
          data: {
            labels: labels,
            datasets: [
              {
                label: "Sales",
                data: data,
                backgroundColor: "rgba(54, 162, 235, 0.5)",
              },
            ],
          },
        });
      }

      function getSalesData(type) {
        $.ajax({
          url: "/analytics/sales/data/",
          method: "GET",
          data: { type: type },
          success: function (response) {
            renderChart(response.labels, response.data);
          },
          error: function () {
            alert("Ocurrió un error");
          },
        });
      }

      $(document).ready(function () {
        const type = $("#salesChart").data("type");
        getSalesData(type);
      });
    </script>
{% end block %}

{% block content' %}
        <div class="row">
                <div class="col-12">
                </h1>
                Ventas
                </h1>
                </div>
        </div>

<div class="row">
<div class="col">
    <p>Recientes</p>
    <ol>
    {% for order in recent_orders %}
    <li>{{order.order_id}}
    {{order.total}}
    {{order.updated}}
    </li>
    {% end for %}
    </ol>
</div>
<div class="col">
    <p>Enviados</p>
    <ol>
    {% for order in shipped_orders %}
    <li>{{order.order_id}}
    {{order.total}}
    {{order.updated}}
    </li>
    {% end for %}
    </ol>
</div>
<div class="col">
    <p>Pagados</p>
    <ol>
    {% for order in paid_orders %}
    <li>{{order.order_id}}
    {{order.total}}
    {{order.updated}}
    </li>
    {% end for %}
    </ol>
</div>
</div>
 <div class="col">
      <canvas
        class="render-chart"
        id="thisWeekSales"
        width="400"
        height="200"
        data-type="week"
      ></canvas>
    </div>
{% end block %}

### config/urls.py

In [ ]:
urlpatterns = [
    ...
    path("analytics", include("analytics.urls"))
]

### analytics/urls.py

In [ ]:
from django.urls import path
from analytics import views

urlpatterns = [
    path("sales", view.SalesView.as_view(), name="sales-analytics")
    path("sales/data", view.SalesAjaxView.as_view(), name="sales-data")
]

- docker exec -it hellodjango-web-1 bash
- python manage.py createsuperuser
- test on analytics/ventas should get the text ventas

## Order

### Create app orders

- python manage.py startapp order

### order/admin.py

In [ ]:
from django.contrib import admin
from .models import Order

admin.site.register(Order)

### order/models.py

In [ ]:
import math
from datetime import datetime, timedelta
from django.conf import settings
from django.db import models
from django.db.models import Count, Sum, Avg
from django.db.models.signals import pre_save. post_save
from django.core.urlresolvers import reverse
from django.utils import timezone

from addresses.models import Adress
form billing.models import BillingProfile
from carts.models import Cart
from ecommerce.utils import unique_order_id_generator
from products.models import Product

ORDER_STATUS_CHOICES = [
    ("created", "Created"), ## Como se guarda en db, Como se muestra al usuario
    ("paid", "Paid"),
    ("shipped", "Shipped")
    ("refunded", "Refunded")
]

class OrderManagerQuerySet(models.query.QuerySet):
    def recent(self):
        return self.order_by("-updated", "-timestamp")
    
    def get_sales_breakdown(self):
        recent = self.recent().not_refunded()
        recent_data = recent.totals_data()
        recent_cart_data = recent.cart_data()
        shipped = recent.by_status(status="shipped")
        shipped_data = shipped.totals_data()
        paid = recent.by_status(status="paid")
        paid_data = paid.totals_data()
        data = {
            "recent": recent,
            "recent_data": recent_data,
            "recent_cart_data": "recent_cart_data,
            "shipped": shipped,
            "shipped_data": shipped_data,
            "paid": paid,
            "paid_data": paid_data
        }


    def by_status(self, status="shipped"):
        return self.filter(status=status)
    
    def not_refunded(self, status="refunded"):
        return self.filter(status=status)
    
    def not_created(self):
        return self.exclude(status="created")
    
    def by_range(self_ start_date, end_date):
        if end_date is None:
            return self.filter(updated__gte=start_date)
        return self.filter(updated__gte=start_date).filter(updated__lte=end_date)
    
    def totals_data(self):
        return aggregate(Sum("total"), Avg("total))
    
    def by_weeks_range(self, weeks_ago=7, number_of_weeks=2):
        if number_of_weeks > weeks_ago:
            number_of_weeks = weeks_ago
        days_ago_start = weeks_ago * 7
        days_ago_end = days_ago_start - number_of_weeks * 7
        start_date = timezone.now() - timedelta(days=days_ago_start)
        end_date = timezone.now() - timedelta(days=days_ago_end)
        return self.by_range(start_date, end_date=end_date)

class OrderManager(models.Manager):
    def get_queryset(self):
        return OrderManagerQuerySet(self.model, using=self._db)
    
    def by_request(self, request: HttpRequest):
        return self.get_queryset().by_request(request)
    
    def new_or_get(self, billing_profile, cart_obj):
        created = False
        qs = self.get_queryset().filter(
            billing_profile=billing_profile,
            cart=cart_obj,
            active=True,
            status="create"
        )
        if qs.count() == 1:
            obj = qs.first()
        else:
            obj = self.model.objects.create(
                billing_profile=billing_profile,
                cart=cart_obj
            )
            created=True
        return obj, created

class Order(models.Model):
    billing_profile = models.ForeignKey(BillingProfile, null=True, blank=True)
    order_id = models.ChartField(max_length=120, blank=True)
    shipping_address = models.ForeignKey(Address, related_name="shipping_address", null=True, blank=True)
    billing_address = models.ForeignKey(Address, related_name="billing_address", null=True, blank=True)
    cart = models.ForeignKey(Cart)
    status = models.ChartField(max_length=120, default="created", choices=ORDER_STATUS_CHOICES)
    shipping_total = models.DecimalField(default=5.99, max_digits=100, decimal_placess=2)
    total = models.DecimalField(default=0.0, max_digits=100, decimal_placess=2)
    active = models.BooleanField(default=True)
    updated = models.DateTimeField(auto_now=True)
    timestamp = models.DateTimeField(auto_new_add=True)

    def __str__(self):
        return self.order_id
    
    objects = OrderManager()

    class Meta:
        ordering = ["-timestamp", "-updated"]

    def get_absolute_url(self):
        return reverse("orders:detail", kargs="order_id": self.order_id)

    def get_status(self):
        if self.status = "refunded"
            return "Refunded order"
        elif self.status = "shipped":
            return "Shipped"
        return "Shipping Soon"





## Addresses

python manage.py startapp addresses

### addresses/models.py

In [ ]:
from django.db import models
from django.core.urlresolvers import reverse
from billing.models import BillingProfile

ADDRESSES_TYPES = 
    (
        ("billing", "Billing address"),
    ("shipping", "Shipping address")
    )

class Address(models.Model):
    billing_profile = models.ForeignKey(BillingProfile)
    name = models.ChartField(max_length=120, null=True, blank=True, help_text="Shipping To? Who is it for?")
    nickname = models.ChartField(max_length=120, null=True, blank=True, help_text="Internal Reference Nickname")
    address_type = models.ChartField(max_length=120, choices=ADDRESSES_TYPES)
    address_line_1 = models.ChartField(max_length=120)
    address_line_2 = models.ChartField(max_length=120, nullt=True, blank=True)
    city = models.ChartField(max_length=120)
    country = models.ChartField(max_length=120, default="Mexico")
    state = models.ChartField(max_length=120)
    postal_code = models.ChartField(max_length=120)

    def __str__(self):
        if self.nickname:
            return str(self.nickname)
        return str(address_line_1)
    
    def get_absolute_url(self):
        return reverse("address-update". kargs=("pk":self.pk))
    
    def get_short_address(self):
        for_name = self.name
        if self.nickname:
            for_name = "{self.nickname} | {for_name}"
        return "{for_name} {self.address_line_1}, {self.city}"
    
    def get_address(self):
        return "{for_name} \n{self.address_line_1}\n {self.address_line_2}\n, {self.country}\n, {self.state}\n, {self.city}\n , {self.postal_code}"




### addresses/admin.py

In [ ]:
from django.contrib import admin
from .models import Addresses

admin.site.register(Addresses)

## Billing

python manage.py startapp billing

### billing/models.py

In [ ]:
from django.conf import settings
from django.db import models
from django.db.models.signals import pre_save. post_save
from django.core.urlresolvers import reverse
from accounts.models import GuestEmail

User = settings.AUTH_USER_MODEL

class BillingProfile(models.Model):
    user = models.OneToOneField(User, null=True, blank=True, on_delete=models.CASCADE)
    email = models.EmailField()
    active = models.BooleandField(default=True)
    update = models.DateTimeField(auto_now=True)
    timestamp = models.DateTimeField(auto_now_add=True)
    customer_id = models.ChartField(max_length=120, null=True, blank=True)



### billing/admin.py

In [ ]:
from django.contrib import admin
from .models import BillingProfile

admin.site.register(BillingProfile)

## Cart

python manage.py startapp cart

### cart/models.py

In [ ]:
form decimal import Decimal
from django.conf import settings
from django.db import models
from django.db.models.signals import pre_save. post_save, m2m_changed

from products.models import Product

User = settings.AUTH_USER_MODEL

class Cart(models.Model):
    user = models.ForeignKey(User, null=True, blank=True)
    products = models.ManyToManyField(defaul=0, max_digits=100, decimal_plcaes=2)
    subtotal = models.ManyToManyField(defaul=0.0, max_digits=100, decimal_plcaes=2)
    products = models.ManyToManyField(defaul=0.0, max_digits=100, decimal_plcaes=2)
    updated = models.DateTimeField(auto_now=True)
    timestamp = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return str(self.id)
    
    @property
    def is_digital(self):
        qs = self.products.all()
        new_qs = qs.filter(is_digital=False)
        if new_qs.exists():
            return False
        return True

carts/admin.py

In [ ]:
from django.contrib import admin
from .models import Carts

admin.site.register(Carts)

## Products

python manage.py startapp products

### products/models.py

In [ ]:
import os
form decimal import Decimal
from django.conf import settings
from django.db import models
from django.db.models.signals import pre_save. post_save, m2m_changed

def upload_image_path(file_path):
    base_name = os.path.basename(file_path)
    name, ext = os.path.splittext(base_name)
    return name, ext

class Product(modles.Model):
    title = models.ChartField(max_length=120)
    slug = models.SlugField(max_length=120, blank=True)
    price = models.DecimalField(decimal_places=2, max_digits=20, default=99.99)
    image = models.ImageField(upload_to=upload_image_path, null=True, blank=True)
    featured = models.BooleanField(default=False)
    active = models.BooleanField(default=True)
    timestamp = models.DateTimeField(autno_now_add=True)
    is_digital = models.BooleanField(default=False)
    description = models.TextField()



### products/admin.py

In [ ]:
from django.contrib import admin
from .models import Product

admin.site.register(Product)

## Accounts

### accounts/apps.py
Same for:
 - order/apps.py
 - products/apps.py
 - address/apps.py
 - cart/apps.py
 - billing/apps.py

In [ ]:
from django.apps import AppConfig


class AccountsConfig(AppConfig):
    name = 'accounts'


### accounts/admin.py

In [ ]:
from django.contrib import admin
from .models import User, GuestEmail

admin.site.register(User)
admin.site.register(GuestEmail)

### accounts/models.py

In [ ]:
from django.db import models
from datetime import timedelta
from django.conf import settings
from django.urls import reverse
from django.db import models
from django.db.models import q
from django.db.models.signals import pre_save, post_save
from django.countrib.auth.models import (AbastractBaseUser, BaseUseManager)
from django.core.mail import send_mail
from django.template.loader import get_template
from django.utils iport timenow

DEFAULT_ACTIVATION_DAYS = getattr(settings, "DEFAULT_ACTIVATION_DAYS", 7)

class UserManager(BaseUseManager):
    def create_user(self, email, full_name=None, password=None, is_active=True, is_staff=False, is_admin=False):
        if not email:
            raise ValueError("Los usuarios deben tener un correo")
        if not password:
            raise ValueError("Los usuarios deben tener un password")
        user_obj = self.model(
            email = self.normalize_email(email)
            full_name = full_name
        )
        user_obj.set_password(password)
        user_obj.staff = is_staff
        user_obj.admin = is_admin
        user_obj.is_active = is_active
        user_obj.save(using=self_db)
        return user_obj
    
    def create_staff_user(self, email, full_name=None, password=None):
        user=self.create_user(
            email,
            full_name=full_name,
            password=password,
            is_staff=True
        )
        return user

   def create_staff_userstar(self, email, full_name=None, password=None):
        user=self.create_user(
            email,
            full_name=full_name,
            password=password,
            is_staff=True
            is_admin=True
        )
        return user

class User(AbastractBaseUser):
    email = models.EmailField(max_length=235, unique=True)
    full_name = models.ChartField(max_length=235, blank=True, null=True)
    is_active = models.BooleanField(default=False)
    staff = models.BooleanField(default=False)
    admin = models.BooleanField(default=False)
    timestamp = models.DateTimeField(auto_now_add=True)

    USERNAME_FIELD = "email"
    REQUIRED_FIELDS = []
    OBJECTS = UserManager[]

    def _str_(self):
        return self.email ## Para mostrarse sea solo el email
    
    def get_full_name(self):
        if self.full_name:
            return self.full_name
        return self.email

    def get_shot_name(self):
        return self.email

    def has_permission(self, perm, obj=None):
        return True
    
    def has_module_params(self, app_label):
        return True ## Para acceso a ciertas apps

    @property
    def is_staff(self):
        if self.is.admin:
            return True
        return self.staff
    
    @property
    def is_admin(self):
        return True
        
class GuestEmail(models.Model):
    email = models.EmailField()
    active = models.BooleanField(default=True)
    update = models.DateTimeField(auto_now=True)
    timestamp = models.DateTimeField(auto_new_add=True)

    def __str__(self):
        return self.email

## Config

### config/settings.py

- add the apps

In [ ]:
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig", 
    "accounts.apps.AccountsConfig", #<--- Added app
    "orders.apps.OrdersConfig", #<--- Added app
    "addresses.apps.AddressesConfig", #<--- Added app
    "analytics.apps.AnalyticsConfig", #<--- Added app
    "billing.apps.BillingConfig", #<--- Added app
    "carts.apps.CartsConfig", #<--- Added app
    "products.apps.ProductsConfig", #<--- Added app
    "forms.apps.FormsConfig", #<--- Added app
    "base.apps.BaseConfig",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]


### config/urls.py

In [ ]:
 urlpatterns = [
    ...
 path("forms/", include("forms.urls"))
 ]

## Prueba de codigo

- docker compose build
- docker compose up

- Para migrar un solo modelo con
    - python manage.py migrate products zero ## Reverte migraciones
    - python manage.py makemigrations
    - python manage.py migrate



## Chart JS

### templates/chart/chart_sales.html

In [ ]:
{% load static %}
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Document</title>
  </head>
  <body>
    <div class="col">
      <p>Enviadas</p>
      <ol>
        {% for order in shipped_orders %}
        <li>
          <p>
            Order ID: {{ order.id }} - Total: {{ order.total }} - Date: {{
            order.updated }}
          </p>
        </li>
        {% endfor %}
      </ol>
    </div>
    <div class="col">
      <p>Pagadas</p>
      <ol>
        {% for order in paid_orders %}
        <li>
          <p>
            Order ID: {{ order.id }} - Total: {{ order.total }} - Date: {{
            order.updated }}
          </p>
        </li>
        {% endfor %}
      </ol>
    </div>
    <div class="col">
      <canvas
        class="render-chart"
        id="salesChart"
        width="400"
        height="200"
        data-type="week"
      ></canvas>
    </div>
    <!-- jQuery -->
    <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
    <!-- Chart.js -->
    <script
      src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/4.5.0/chart.min.js"
      integrity="sha512-n/G+dROKbKL3GVngGWmWfwK0yPctjZQM752diVYnXZtD/48agpUKLIn0xDQL9ydZ91x6BiOmTIFwWjjFi2kEFg=="
      crossorigin="anonymous"
      referrerpolicy="no-referrer"
    ></script>
    <script>
      function renderChart(labels, data) {
        const ctx = document.getElementById("salesChart").getContext("2d");
        new Chart(ctx, {
          type: "polarArea",
          data: {
            labels: labels,
            datasets: [
              {
                label: "Sales",
                data: data,
                backgroundColor: "rgba(54, 162, 235, 0.5)",
              },
            ],
          },
        });
      }

      function getSalesData(type) {
        $.ajax({
          url: "/analytics/sales/data/",
          method: "GET",
          data: { type: type },
          success: function (response) {
            renderChart(response.labels, response.data);
          },
          error: function () {
            alert("Ocurrió un error");
          },
        });
      }

      $(document).ready(function () {
        const type = $("#salesChart").data("type");
        getSalesData(type);
      });
    </script>
  </body>
</html>
{% endblock %}

### JQuery

In [ ]:
<script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>

## Forms

### forms.py

In [ ]:
from django import forms

class TestForm(forms.form):
    some_text = forms.CharField(label="Ingresa un Texto", widget=forms.Textarea(attrs="rows":4, "cols":10))
    ## widget=forms.Textarea(attrs="rows":4, "cols":10) hace un textArea de 10x4
    ## hay diferentes tipos de widgets, por ejemplo fecha se puede configurar
    ## YEARS = [x for x in range(1920, 2030)]
    date = forms.DateField(widget=forms.SelectDateWidget(years=YEARS)) 
    boolean = forms.BooleanField()
    integer = forms.IntegerField()
    email = forms.EmailField()
    ## Otras formas
    ## MY_CHOICE = [
        ("db-value1", "Opcion 1"),
        ("db-value2", "Opcion 2"),
        ("db-value3", "Opcion 3")
    ]
    opciones = forms.ChartField(label="Selecciona una opcion", widget=forms.Select(choices=MY_CHOICE))
    opciones_radio = forms.ChartField(label="Selecciona una opcion", widget=forms.RadioSelect(choices=MY_CHOICE))
    opciones_checkbox = forms.ChartField(label="Selecciona una opcion", widget=forms.CheckboxSelectMultiple(choices=MY_CHOICE))

    def clean_entero(self, *args: Any, **kargs: Any):
        entero = self.cleaned_data.get("entero")
        if entero > 10:
            raise forms.ValidationError("El numero deber ser menor a 10")
        return entero

    def clean_texto(self, *args: Any, **kargs: Any):
        texto = self.cleaned_data.get("texto")
        if texto.length > 10:
            raise forms.ValidationError("El texto deber ser menor o igual a 10 caracteres")
        return texto

## Model Form
from .models import Product

labels = {
    "title" : "Mi etiqueta para el titulo",
    "slug" : "Mi etiqueta para el slug",
    "price" : "Mi etiqueta para el price",
}


class ProductModelForm(forms.ModelForm):
    class Meta:
        model = Product
        fields = [
            "title",
            "slug",
            "price"
        ]
        exclude = []
    
    def clean_title(self, *args, **kargs):
        title= self.cleaned_data.get("title")
        if len(title)<10:
            raise forms.ValidationError("El titulo debe tener mas de 10 caracteres")
        return title
    
    def clean_slug(self, *args, **kargs):
        slug= self.cleaned_data.get("slug")
        if len(slug)<10:
            raise forms.ValidationError("El slug debe tener mas de 10 caracteres")
        return slug

## Model formset



### forms/views.py

In [ ]:
form django.shortcuts import render
form .forms import TestForm
form django.forms import formset_factory, modelformset_factory

def home(request: HttpRequest):
    
    initial_data = {
    "some_text": "Texto inicial",
    "boolean":  True,
    "integer" :  100,
    "email": "test@gmail.com"
    }

    form = TestForm(request.POST or None, initial=initial_data)
    if form.is_valid():
        print(form.cleaned_data) #<--- Imprime on docker console
    return render(request, "forms.html", {"form": form})

### WITH FORM SETS
def home(request: HttpRequest):
    TestFormSet = formset_factory(TestForm, extra=3)
    formset = TestFormSet(request.POST or None)
    for form in formset:
        print(form.cleaned_data)
    context = {
        "formset" = formset
    }
    return render(request, "formset_view.html", context)

### WITH MODELFORM SETS
def home(request: HttpRequest):
    TestFormSet = modelformset_factory(Product, form=ProductModelForm)
    formset = ProductModelFormSet(request.POST or None, queryset=Product.objects.all())

    print("formset.data")
    print(formset.data)

    print("formset.errors")
    print(formset.errors)

    formset.clean()
    if formset.is_valid():
        print("ModelFormSet is valid")
        formset.save()

    context = {
        "formset" = formset
    }
    return render(requ  est, "formset_view.html", context)



### forms.html

In [ ]:
<form method="POST" action="">
    {% csrf_token}
    {{form.as_p}}
    <input type="submit" value="Guardar">
    {% if delete_url %}
        <a href="{{delete_url}}">Eliminar</a>
</form>

### forms/urls.py

In [ ]:
import .views import Home

urlpatterns = [
    path("admin/", admin.site.urls)
    path("", home)
]

### forms/apps.py

In [ ]:
form django.apps import AppConfig

class FormConfig(AppConfig):
    default_auto_field = "django.db.models.BigAutoField"
    name = "forms"

### forms/models.py

In [ ]:
from django.db import models
from django.conf import settings

class ProductModel(models.Model):
    title = models.ChartField(max_length=120)
    slug = models.SlugField(unique=True)
    price = models.FloatField()

### forms/admin.py

In [ ]:
from django.config import admin
from .models import ProductModel

admin.site.register(ProductModel)


### FormSets

Grupo de forms

- templates/base.html

In [ ]:
<body>
<div class="container">
{% block content %}
{% endblock _5}
</div>
</body>

- templates/formset_view.html

In [ ]:
{% extends base html %}
{% block content %}

<h1>Form Set</h1>
<form action="" method=POST">
    {{% csrf_token %}}

    {{ formset.management_form}}
    {{ for form in formset %}}
        <div>
            {{ form.as_p}}
            <br>
        </div>
    {{end for}}
<input type="submit" value="Guardar">
</form>

{{% end block %}}